### Import our dependencies

In [1]:
import pandas as pd
import matplotlib as plt
from sklearn.datasets import make_blobs
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import numpy as np
import os
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mc
import statsmodels.formula.api as sm
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import json

### Read our CSV

In [2]:
us_df = pd.read_csv('USvideos.csv')

In [3]:
us_df.head(10)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
5,gHZ1Qz0KiKM,17.14.11,2 Weeks with iPhone X,iJustine,28,2017-11-13T19:07:23.000Z,"ijustine|""week with iPhone X""|""iphone x""|""appl...",119180,9763,511,1434,https://i.ytimg.com/vi/gHZ1Qz0KiKM/default.jpg,False,False,False,Using the iPhone for the past two weeks -- her...
6,39idVpFF7NQ,17.14.11,Roy Moore & Jeff Sessions Cold Open - SNL,Saturday Night Live,24,2017-11-12T05:37:17.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",2103417,15993,2445,1970,https://i.ytimg.com/vi/39idVpFF7NQ/default.jpg,False,False,False,Embattled Alabama Senate candidate Roy Moore (...
7,nc99ccSXST0,17.14.11,5 Ice Cream Gadgets put to the Test,CrazyRussianHacker,28,2017-11-12T21:50:37.000Z,"5 Ice Cream Gadgets|""Ice Cream""|""Cream Sandwic...",817732,23663,778,3432,https://i.ytimg.com/vi/nc99ccSXST0/default.jpg,False,False,False,Ice Cream Pint Combination Lock - http://amzn....
8,jr9QtXwC9vc,17.14.11,The Greatest Showman | Official Trailer 2 [HD]...,20th Century Fox,1,2017-11-13T14:00:23.000Z,"Trailer|""Hugh Jackman""|""Michelle Williams""|""Za...",826059,3543,119,340,https://i.ytimg.com/vi/jr9QtXwC9vc/default.jpg,False,False,False,"Inspired by the imagination of P.T. Barnum, Th..."
9,TUmyygCMMGA,17.14.11,Why the rise of the robots won’t mean the end ...,Vox,25,2017-11-13T13:45:16.000Z,"vox.com|""vox""|""explain""|""shift change""|""future...",256426,12654,1363,2368,https://i.ytimg.com/vi/TUmyygCMMGA/default.jpg,False,False,False,"For now, at least, we have better things to wo..."


In [4]:
# Remove unecessary columns
del_col = ['video_id','thumbnail_link','comments_disabled','ratings_disabled','video_error_or_removed','description']
us_df = us_df.drop(del_col, axis=1)

### Read json file 

In [5]:
cat_json = pd.read_json("US_category_id.JSON")
with open("US_category_id.JSON") as f:
    data = json.load(f)
    print(data)

{'kind': 'youtube#videoCategoryListResponse', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"', 'items': [{'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"', 'id': '1', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Film & Animation', 'assignable': True}}, {'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"', 'id': '2', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Autos & Vehicles', 'assignable': True}}, {'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"', 'id': '10', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Music', 'assignable': True}}, {'kind': 'youtube#videoCategory', 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-oBJavSGkfDI"', 'id': '15', 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'title': 'Pets & Animals', 'assignable': True}}, {'kind': 'yout

In [6]:
cat_dict = [{'id': item['id'], 'title': item['snippet']['title']} for item in cat_json['items']]

In [7]:
us_cat_df = pd.DataFrame(cat_dict)
categories = us_cat_df.rename(index=str, columns={"id": "category_id", "title": "category"})
categories.head(3)

,category_id,category
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music


In [8]:
del_col = ['trending_date','title','channel_title','publish_time','tags']
us_df = us_df.drop(del_col, axis=1)

In [9]:
us_df.category_id.unique()

array([22, 24, 23, 28,  1, 25, 17, 10, 15, 27, 26,  2, 19, 20, 29, 43],
      dtype=int64)

In [10]:
new_us = us_df.groupby(['category_id']).count()

In [11]:
new_us

,views,likes,dislikes,comment_count
category_id,,,,
1,2345,2345,2345,2345
2,384,384,384,384
10,6472,6472,6472,6472
15,920,920,920,920
17,2174,2174,2174,2174
19,402,402,402,402
20,817,817,817,817
22,3210,3210,3210,3210
23,3457,3457,3457,3457


In [12]:
Film_Animation = us_df[us_df['category_id']==1]
People_Blogs = us_df[us_df['category_id']==22]
Entertainment = us_df[us_df['category_id']==24]
Comedy = us_df[us_df['category_id']==23]
Science_Technology = us_df[us_df['category_id']==28]
News_Politics = us_df[us_df['category_id']==25]
Sports = us_df[us_df['category_id']==17]
Music = us_df[us_df['category_id']==10]
Pets_Animals = us_df[us_df['category_id']==15]
Education = us_df[us_df['category_id']==27]
Howto_Style = us_df[us_df['category_id']==26]
Auto_Vehicles = us_df[us_df['category_id']==2]
Travel_Events = us_df[us_df['category_id']==19]
Gaming = us_df[us_df['category_id']==20]
Nonprofits_Activism = us_df[us_df['category_id']==29]
Shows = us_df[us_df['category_id']==43]

In [13]:
fa_views = Film_Animation['views'].sum()
pb_views = People_Blogs['views'].sum()
e_views = Entertainment['views'].sum()
c_views = Comedy['views'].sum()
st_views = Science_Technology['views'].sum()
np_views = News_Politics['views'].sum()
s_views = Sports['views'].sum()
m_views = Music['views'].sum()
pa_views = Pets_Animals['views'].sum()
ed_views = Education['views'].sum()
ht_views = Howto_Style['views'].sum()
av_views = Auto_Vehicles['views'].sum()
te_views = Travel_Events['views'].sum()
g_views = Gaming['views'].sum()
na_views = Nonprofits_Activism['views'].sum()
sh_views = Shows['views'].sum()

In [14]:
fa_likes = Film_Animation['likes'].sum()
pb_likes = People_Blogs['likes'].sum()
e_likes = Entertainment['likes'].sum()
c_likes = Comedy['likes'].sum()
st_likes = Science_Technology['likes'].sum()
np_likes = News_Politics['likes'].sum()
s_likes = Sports['likes'].sum()
m_likes = Music['likes'].sum()
pa_likes = Pets_Animals['likes'].sum()
ed_likes = Education['likes'].sum()
ht_likes = Howto_Style['likes'].sum()
av_likes = Auto_Vehicles['likes'].sum()
te_likes = Travel_Events['likes'].sum()
g_likes = Gaming['likes'].sum()
na_likes = Nonprofits_Activism['likes'].sum()
sh_likes = Shows['likes'].sum()

In [15]:
fa_dislikes = Film_Animation['dislikes'].sum()
pb_dislikes = People_Blogs['dislikes'].sum()
e_dislikes = Entertainment['dislikes'].sum()
c_dislikes = Comedy['dislikes'].sum()
st_dislikes = Science_Technology['dislikes'].sum()
np_dislikes = News_Politics['dislikes'].sum()
s_dislikes = Sports['dislikes'].sum()
m_dislikes = Music['dislikes'].sum()
pa_dislikes = Pets_Animals['dislikes'].sum()
ed_dislikes = Education['dislikes'].sum()
ht_dislikes = Howto_Style['dislikes'].sum()
av_dislikes = Auto_Vehicles['dislikes'].sum()
te_dislikes = Travel_Events['dislikes'].sum()
g_dislikes = Gaming['dislikes'].sum()
na_dislikes = Nonprofits_Activism['dislikes'].sum()
sh_dislikes = Shows['dislikes'].sum()

In [16]:
youtube_summary = pd.DataFrame({'category_id':['Film&Animation', 'Peope&Blogs','Entertainment','Comedy', 'Science&Technology','News&Politics','Sports','Music','Pets&Animals','Education','Howto&Style','Auto&Vehicles','Travel&Events','Gaming','Nonprofits&Activism','Shows'],
                                'Total Views':[fa_views,pb_views,e_views,c_views,st_views,np_views,s_views,m_views,pa_views,ed_views,ht_views,av_views,te_views,g_views,na_views,sh_views],
                                'Total Likes':[fa_likes,pb_likes,e_likes,c_likes,st_likes,np_likes,s_likes,m_likes,pa_likes,ed_likes,ht_likes,av_likes,te_likes,g_likes,na_likes,sh_likes],
                                'Total Dislikes':[fa_dislikes,pb_dislikes,e_dislikes,c_dislikes,st_dislikes,np_dislikes,s_dislikes,m_dislikes,pa_dislikes,ed_dislikes,ht_dislikes,av_dislikes,te_dislikes,g_dislikes,na_dislikes,sh_dislikes]})

In [17]:
youtube_summary

,category_id,Total Views,Total Likes,Total Dislikes
0,Film&Animation,7284156721,165997476,6075148
1,Peope&Blogs,4917191726,186615999,10187901
2,Entertainment,20604388195,530516491,42987663
3,Comedy,5117426208,216346746,7230391
4,Science&Technology,3487756816,82532638,4548402
5,News&Politics,1473765704,18151033,4180049
6,Sports,4404456673,98621211,5133551
7,Music,40132892190,1416838584,51179008
8,Pets&Animals,764651989,19370702,527379
9,Education,1180629990,49257772,1351972


In [18]:
video_count = [2345, 3210,9964,3457,2401,2487,2174,6472,920,1656,4146,384,402,817,57,57 ]
youtube_summary['Video_Count'] = video_count
youtube_summary.sort_values(by='Video_Count', ascending=False)

,category_id,Total Views,Total Likes,Total Dislikes,Video_Count
2,Entertainment,20604388195,530516491,42987663,9964
7,Music,40132892190,1416838584,51179008,6472
10,Howto&Style,4078545064,162880075,5473899,4146
3,Comedy,5117426208,216346746,7230391,3457
1,Peope&Blogs,4917191726,186615999,10187901,3210
5,News&Politics,1473765704,18151033,4180049,2487
4,Science&Technology,3487756816,82532638,4548402,2401
0,Film&Animation,7284156721,165997476,6075148,2345
6,Sports,4404456673,98621211,5133551,2174
9,Education,1180629990,49257772,1351972,1656


In [19]:
youtube_summary
plt.figure(figsize=(200,90))
xlabel = (youtube_summary['category_id'])
ylabel = (youtube_summary['Total Views'])
plt.bar(xlabel,ylabel, color='green')
plt.xlabel('Categories',fontsize=120)
plt.ylabel('Numeber of Views',fontsize=120)
plt.title('Total Views by Category',fontsize=150)
plt.xticks(fontsize=120, rotation=45)
plt.show()

In [20]:
plt.figure(figsize=(200,90))
xlabel = (youtube_summary['category_id'])
ylabel = (youtube_summary['Total Likes'])
plt.bar(xlabel,ylabel, color='dodgerblue')
plt.xlabel('Categories',fontsize=120)
plt.ylabel('Numeber of Likes',fontsize=120)
plt.title('Total Likes by Category',fontsize=150)
plt.xticks(fontsize=120, rotation=45)
plt.show()

In [21]:
plt.figure(figsize=(200,90))
xlabel = (youtube_summary['category_id'])
ylabel = (youtube_summary['Total Dislikes'])
plt.bar(xlabel,ylabel, color='tomato')
plt.xlabel('Categories',fontsize=120)
plt.ylabel('Numeber of Dislikes',fontsize=120)
plt.title('Total Dislikes by Category',fontsize=150)
plt.xticks(fontsize=120, rotation=45)
plt.show()